In [1]:
import numpy as np
import pandas as pd 
from sgd_regression import * 

In [2]:
def load_data(path):
    
    data = pd.read_csv(path)
    
    indice = int(data.shape[0] * 0.7)
    
    shuffle_data = np.random.permutation(data)

    treino = shuffle_data[0:indice, :] 
    teste =  shuffle_data[indice:,:]
    
    X_treino = treino[:,:-1]
    y_treino = treino[:, -1]
    
    X_teste = teste[:,:-1]
    y_teste = teste[:, -1]
    
    return X_treino, y_treino, X_teste, y_teste

In [3]:
path = 'dataset_vehicle.csv'
X_treino, y_treino, X_teste, y_teste = load_data(path)

In [4]:
X_treino.shape[0] + X_teste.shape[0]

846

In [5]:
def separa_classes(X_treino, y_treino):
    vans = []
    opels = []
    saabs = []
    buss = []
    for i in range(len(y_treino)):
        if(y_treino[i] == 'van'):
            vans.append(X_treino[i])
        elif y_treino[i] == 'opel':
            opels.append(X_treino[i])
        elif y_treino[i] == 'bus':
            buss.append(X_treino[i])
        else:
            saabs.append(X_treino[i])
            
    return np.array(vans), np.array(opels), np.array(saabs), np.array(buss)

opel, saab, bus, van = separa_classes(X_treino, y_treino)

In [6]:
def models(opel, saab, bus, van):
    
    opsa = np.concatenate( (opel, saab), axis=0)
    opbs = np.concatenate( (opel, bus), axis=0)
    opvn = np.concatenate( (opel, van), axis=0)
    sabu = np.concatenate( (saab, bus), axis=0)
    savn = np.concatenate( (saab, van), axis=0)
    buva = np.concatenate( (bus, van), axis=0)

    label_opel = np.ones( (opel.shape[0]) )
    label_saab = np.zeros( (saab.shape[0]) )
    label_opsa = np.concatenate( (label_opel, label_saab) )

    label_opel = np.ones( (opel.shape[0]) )
    label_bus = np.zeros( (bus.shape[0]) )
    label_opbs = np.concatenate( (label_opel, label_bus) )

    label_opel = np.ones( (opel.shape[0]) )
    label_van = np.zeros( (van.shape[0]) )
    label_opvn = np.concatenate( (label_opel, label_van) )

    label_saab = np.ones( (saab.shape[0]) )
    label_bus = np.zeros( (bus.shape[0]) )
    label_sabu = np.concatenate( (label_saab, label_bus) )

    label_saab = np.ones( (saab.shape[0]) )
    label_van = np.zeros( (van.shape[0]) )
    label_savn = np.concatenate( (label_saab, label_van) ) 

    label_bus = np.ones( (bus.shape[0]) )
    label_van = np.zeros( (van.shape[0]) )
    label_buva = np.concatenate( (label_bus, label_van) )

    opel_saab = SGDRegression()
    opel_saab.fit(opsa, label_opsa)
    
    opel_bus = SGDRegression()
    opel_bus.fit(opbs, label_opbs)
    
    opel_van = SGDRegression()
    opel_van.fit(opvn, label_opvn)
    
    saab_bus = SGDRegression()
    saab_bus.fit(sabu, label_sabu)
    
    saab_van = SGDRegression()
    saab_van.fit(savn, label_savn)
    
    bus_van = SGDRegression()
    bus_van.fit(buva, label_buva)

In [7]:
#models(opel, saab, bus, van)

In [19]:
def votacao(label):
    
    if label == "van":
        if pred[2] +  pred[4] + pred[5] < 2:
            return 1
        else:
            return 0
    elif label == "opel":
        if pred[0] +  pred[1] + pred[2] > 1:
            return 1
        else:
            return 0
    elif label == "bus":
        if pred[1] +  pred[3] + pred[5] < 2:
            return 1
        else:
            return 0
    else:
        if pred[2] +  pred[4] + pred[5] < 2:
            return 1
        else:
            return 0

In [20]:
data_vehicle = pd.read_csv('dataset_vehicle.csv')
data_vehicle.shape

(846, 19)

In [21]:
#Breaking the data into training and test set
import random
def train_test_split(data, split, trainingSet = [], testSet = []):
    for x in range(len(data)):
        if random.random() < split:
            trainingSet.append(data[x])
        else:
            testSet.append(data[x])

In [22]:
df = data_vehicle.values.tolist()
trainingSet = []
testSet = []
split = 0.7
train_test_split(df, split, trainingSet, testSet)

In [23]:
#Define Euclidean distances
import math
def Euclideandist(x,xi, length):
    d = 0.0
    for i in range(length):
        d += pow(float(x[i])- float(xi[i]),2)
    return math.sqrt(d)

In [13]:
confusion_matrix = pd.DataFrame(
    {
        "bus": [0, 0, 0, 0],
        "van": [0, 0, 0, 0],
        "opel": [0, 0, 0, 0],
        "saab": [0, 0, 0, 0],
    }
)

confusion_matrix = confusion_matrix.rename(index = {0: "bus", 1: "van", 2: "opel", 3: "saab"})
confusion_matrix

,bus,van,opel,saab
bus,0,0,0,0
van,0,0,0,0
opel,0,0,0,0
saab,0,0,0,0


In [14]:
#Getting the K neighbours having the closest Euclidean distance to the test instance
import operator
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = Euclideandist(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

#After sorting the neighbours based on their respective classes, max voting to give the final class of the test instance
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)#Sorting it based on votes
    return sortedVotes[0][0] #Please note we need the class for the top voted class, hence [0][0]#

#Getting the accuracy
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

# generate predictions
predictions=[]
k = 3
for x in range(len(testSet)):
    neighbors = getNeighbors(trainingSet, testSet[x], k)
    result = getResponse(neighbors)
    if testSet[x][-1] == 'bus':
        confusion_matrix.iloc[0][result]+=1
    elif testSet[x][-1] == 'van':
        confusion_matrix.iloc[1][result]+=1
    elif testSet[x][-1] == 'opel':
        confusion_matrix.iloc[2][result]+=1
    else:
        confusion_matrix.iloc[3][result]+=1
    predictions.append(result)
    print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))

> predicted='saab', actual='van'
> predicted='van', actual='bus'
> predicted='van', actual='van'
> predicted='van', actual='van'
> predicted='saab', actual='saab'
> predicted='van', actual='van'
> predicted='van', actual='saab'
> predicted='van', actual='van'
> predicted='bus', actual='bus'
> predicted='saab', actual='van'
> predicted='opel', actual='saab'
> predicted='van', actual='saab'
> predicted='saab', actual='opel'
> predicted='bus', actual='bus'
> predicted='van', actual='van'
> predicted='van', actual='van'
> predicted='van', actual='van'
> predicted='bus', actual='bus'
> predicted='opel', actual='saab'
> predicted='bus', actual='bus'
> predicted='van', actual='van'
> predicted='saab', actual='opel'
> predicted='bus', actual='bus'
> predicted='bus', actual='opel'
> predicted='saab', actual='bus'
> predicted='opel', actual='opel'
> predicted='van', actual='opel'
> predicted='bus', actual='saab'
> predicted='saab', actual='bus'
> predicted='bus', actual='bus'
> predicted='van', 

In [15]:
confusion_matrix

,bus,van,opel,saab
bus,53,3,0,11
van,1,51,0,3
opel,3,10,25,20
saab,6,6,16,25


In [16]:
accuracy = getAccuracy(testSet, predictions)
print('Accuracy: ' + repr(accuracy) + '%')

Accuracy: 66.09442060085837%


In [17]:
from sklearn.neighbors import KNeighborsClassifier

colnames_numeric = data_vehicle.columns[0:18]

trainingSet2 = pd.DataFrame(np.array(trainingSet).reshape(len(trainingSet),19), columns = data_vehicle.columns)
testSet2 = pd.DataFrame(np.array(testSet).reshape(len(testSet),19), columns = data_vehicle.columns)

trainingSet2[colnames_numeric] = trainingSet2[colnames_numeric].apply(pd.to_numeric, errors = 'coerce', axis = 0)
testSet2[colnames_numeric] = testSet2[colnames_numeric].apply(pd.to_numeric, errors = 'coerce', axis = 0)

knn_sklearn = KNeighborsClassifier(n_neighbors = 1, metric='euclidean')
x_train,y_train = trainingSet2.loc[:,trainingSet2.columns != 'Class'], trainingSet2.loc[:,'Class']
x_test,y_test = testSet2.loc[:,testSet2.columns != 'Class'], testSet2.loc[:,'Class']
knn_sklearn.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='euclidean',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [18]:
prediction = knn_sklearn.predict(x_test)
print('Prediction: {}'.format(prediction))
print('With KNN (K=3) accuracy is: ',knn_sklearn.score(x_test,y_test)) # accuracy

Prediction: ['saab' 'van' 'van' 'van' 'saab' 'van' 'van' 'van' 'bus' 'opel' 'saab'
 'saab' 'saab' 'bus' 'van' 'van' 'van' 'bus' 'opel' 'bus' 'van' 'saab'
 'bus' 'bus' 'opel' 'saab' 'van' 'saab' 'bus' 'bus' 'van' 'opel' 'saab'
 'van' 'van' 'bus' 'van' 'van' 'bus' 'saab' 'van' 'van' 'bus' 'saab'
 'opel' 'saab' 'opel' 'saab' 'saab' 'bus' 'bus' 'saab' 'saab' 'van' 'saab'
 'saab' 'bus' 'bus' 'bus' 'bus' 'van' 'van' 'bus' 'opel' 'bus' 'saab'
 'saab' 'opel' 'van' 'saab' 'bus' 'saab' 'van' 'bus' 'opel' 'van' 'opel'
 'van' 'saab' 'van' 'van' 'bus' 'saab' 'van' 'van' 'van' 'opel' 'saab'
 'opel' 'opel' 'bus' 'saab' 'bus' 'van' 'opel' 'van' 'opel' 'bus' 'bus'
 'van' 'bus' 'opel' 'bus' 'saab' 'opel' 'van' 'opel' 'bus' 'van' 'bus'
 'van' 'van' 'bus' 'bus' 'bus' 'saab' 'van' 'van' 'opel' 'van' 'van'
 'saab' 'opel' 'saab' 'van' 'van' 'opel' 'opel' 'saab' 'van' 'opel' 'bus'
 'bus' 'opel' 'bus' 'van' 'saab' 'opel' 'bus' 'saab' 'bus' 'saab' 'van'
 'opel' 'saab' 'opel' 'saab' 'van' 'bus' 'saab' 'opel' 'sa